In [1]:
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras_tuner as kt
from keras.src.utils.module_utils import tensorflow

from tensorflow.keras import models, layers, Input, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

from sklearn.feature_extraction.text import CountVectorizer

from Code.utils.dataset import Dataset

import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

[nltk_data] Downloading package punkt to C:\Users\Guilherme
[nltk_data]     Barbosa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
np.random.seed(42)
tensorflow.random.set_seed(42)
random.seed(42)

In [3]:
# Load datasets

dataset = Dataset('../../Dataset/DatasetsGerados/dataset_training_input.csv',
                  '../../Dataset/DatasetsGerados/dataset_training_output.csv',
                  '../../Dataset/DatasetsGerados/dataset_validation_input.csv',
                  '../../Dataset/DatasetsGerados/dataset_validation_output.csv',
                  '../../Dataset/DatasetsGerados/dataset_test_input.csv',
                  '../../Dataset/DatasetsGerados/dataset_test_output.csv')

# Remover pontuação deu pior resultado
X_train, y_train, X_validation, y_validation, X_test, y_test, ids = dataset.get_datasets_unprocessed('Text', 'Label', sep='\t', rem_punctuation=False)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', num_labels=1) # num_labels makes the output 1D

X_train = tokenizer(X_train['Text'].tolist(), return_tensors='tf', padding=True, truncation=True)
X_validation = tokenizer(X_validation['Text'].tolist(), return_tensors='tf', padding=True, truncation=True)
X_test = tokenizer(X_test['Text'].tolist(), return_tensors='tf', padding=True, truncation=True)

In [4]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
history = model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_validation,y_validation))

# Plot curves

history_dict = history.history
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']
epochs = range(1, len(acc_values) + 1)

plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()

# Plot loss curves

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [5]:
# Predict output

results = model(X_test)
output = tf.sigmoid(results.logits)

print(output)

if y_test is not None:
    # Evaluate Model

    print(model.evaluate(X_test, y_test))

KeyboardInterrupt: 

In [ ]:
# Store output

results_filepath = './Results/llm_results.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(results_filepath), exist_ok=True)

results = dataset.merge_results(ids, output)
results.to_csv(results_filepath, sep='\t', index=False)

In [ ]:
dataset.set_dataset_test('../../Dataset/dataset1_inputs.csv', '../../Dataset/dataset1_outputs.csv')

X_test, y_test, ids = dataset.get_test_dataset_unprocessed('Text', 'Label', sep='\t', rem_punctuation=False)

X_test = tokenizer(X_test, return_tensors='tf', padding=True, truncation=True)